In [12]:
import sys
!{sys.executable} -m pip install kfp >/dev/null

You should consider upgrading via the 'pip install --upgrade pip' command.


In [13]:
import os, json
import kfp
import kfp.compiler as compiler
import random
import string

generate = lambda hint: "{}-{}".format(hint, ''.join([random.choice(string.digits) for n in range(4)]))

In [14]:
existing_token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=existing_token)

# Input and output parameters for the pipeline run

In [15]:
# Project owner resources
project_id = "pjti25"
project_name = "titanic-3"
project_owner = "ravih1"
ptrain_dataset = f'titanic-train:{project_owner}'
ptest_dataset = f'titanic-test:{project_owner}'

# User specific resources
train_fs = 'titanic-train-fs-user'
test_fs = 'titanic-test-fs-user'
training_program = 'titanic-code-user'
model = 'titanic-model-user'

# Program specific variables
image = "docker.io/ocdr/dkube-datascience-tf-cpu:fs-v2.0.0"
dataset_mount_points = ["/dataset/train", "/dataset/test"]
output_featureset_mount_points = ["/opt/dkube/output/train", "/opt/dkube/output/test"]
preprocessing_script = f"python preprocessing.py --train_fs {train_fs} --test_fs {test_fs}"
training_script = "python training.py"
train_inp_mount_points = ["/titanic-train","/titanic-test"]
train_out_mount_points = ["/model"]

user = os.getenv('USERNAME')

# Loading components

In [16]:
components_url = "https://raw.githubusercontent.com/oneconvergence/dkube/master/components/"
dkube_preprocessing_op = kfp.components.load_component_from_url(components_url + "preprocess/component.yaml")
dkube_training_op = kfp.components.load_component_from_url(components_url + "training/component.yaml")
dkube_storage_op  = kfp.components.load_component_from_url(components_url + "storage/component.yaml")
dkube_submit_op = kfp.components.load_component_from_url(components_url + "submit/component.yaml")

# Pipeline Definition

In [17]:
@kfp.dsl.pipeline(
    name='dkube-titanic-pl',
    description='example titanic pipeline to submit to leaderboard'
)
def titanic_pipeline(token, project_id):

    preprocessing = dkube_preprocessing_op(token, json.dumps({"image": image}),
                                           tags=json.dumps([f"project:{project_id}"]),
                                           program=training_program, run_script=preprocessing_script,
                                           datasets=json.dumps([ptrain_dataset, ptest_dataset]), 
                                           output_featuresets=json.dumps([train_fs, test_fs]),
                                           input_dataset_mounts=json.dumps(dataset_mount_points), 
                                           output_featureset_mounts=json.dumps(output_featureset_mount_points)
                                            )

    with kfp.dsl.ExitHandler(exit_op=dkube_storage_op("reclaim", token, namespace="kubeflow",uid="{{workflow.uid}}")):
        input_volumes = json.dumps([
                                    "{{workflow.uid}}-model@model://" + model,
                                    "{{workflow.uid}}-code@program://" + training_program
                                    ])
        train       = dkube_training_op(token, json.dumps({"image": image}),
                                        tags=json.dumps([f"project:{project_id}"]),
                                        framework="sklearn", version="0.23.2",
                                        program=training_program, run_script=training_script,
                                        featuresets=json.dumps([train_fs, test_fs]), outputs=json.dumps([model]),
                                        input_featureset_mounts=json.dumps(train_inp_mount_points),
                                        output_mounts=json.dumps(train_out_mount_points)).after(preprocessing)
        
        storage  = dkube_storage_op("export", token, namespace="kubeflow", input_volumes=input_volumes).after(train)

    
        predict_op = kfp.dsl.ContainerOp(
            name="predict",
            image=image,
            command=["python", "/code/predict.py"],
            pvolumes={
                     "/model/": kfp.dsl.PipelineVolume(pvc="{{workflow.uid}}-model"),
                     "/code/": kfp.dsl.PipelineVolume(pvc="{{workflow.uid}}-code")
                     },
            file_outputs={"output": "/tmp/prediction.csv"},
        ).after(storage)

        predictions = kfp.dsl.InputArgumentPath(predict_op.outputs["output"])

        submit = dkube_submit_op(token, project_id, predictions=predict_op.outputs["output"]).after(predict_op)

# Compling pipeline into tar file

In [18]:
pl_file_name = 'train_pl.tar.gz'
pl_name = f'[{project_name}]' + f'-{user}-' + generate('pl') 

compiler.Compiler().compile(titanic_pipeline, pl_file_name)
# Upload this generated tarball into the Pipelines UI

# Uploading Pipeline

In [19]:
client = kfp.Client(existing_token=existing_token)

pipeline_id = client.get_pipeline_id(pl_name)
try:
    resp = client.upload_pipeline(pipeline_package_path = pl_file_name, 
                                  pipeline_name = pl_name, description = None)
except BaseException as e:
    print(e)

# Creating experiment

In [20]:
experiment_name = f'[{project_name}]' + f'-{user}-' + 'exp'
try:
    titanic_experiment = client.create_experiment(name=experiment_name)
except BaseException as e:
    print(e)

# Creating run from pipeline under the titanic experiment

In [21]:
run_name = f"[{project_name}]" + f'-{user}-' + generate("run")
try:
    pipeline_id = client.get_pipeline_id(pl_name)
    run = client.run_pipeline(titanic_experiment.id, run_name, None, pipeline_id=pipeline_id,
                              params={"token":existing_token, "project_id":project_id})
except BaseException as e:
    print(e)


In [22]:
# client = kfp.Client(existing_token=existing_token)
# #Replace [titanic] & wprz8s with your project name and id respectively.
# client.create_run_from_pipeline_func(titanic_pipeline, run_name="[titanic] Run" + str(runid), arguments={"token":existing_token,"project_id":"ynli7c"}, experiment_name = titanic_experiment.name)
# runid += 1